# Introduction
This notebook performs the tasks in the exercise

In [82]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from migrator import Migrator
from database import Database
from environs import Env
import tasks

from tasks import Task

ModuleNotFoundError: No module named 'tasks.Task'; 'tasks' is not a package

## Part 1
Setting up and seeding the database.

In [48]:
env = Env()
env.read_env(".env")

[autoreload of tasks failed: Traceback (most recent call last):
  File "/Users/larswaage/Code/large-distributed-data/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/larswaage/Code/large-distributed-data/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/larswaage/Code/large-distributed-data/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/larswaage/Code/large-distributed-data/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/larswaage/Code/large-distributed-data/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b

In [19]:
database = Database(
    host=env.str("DB_HOST"),
    port=env.int("DB_PORT"),
    user=env.str("DB_USER"),
    password=env.str("DB_PASSWORD"),
    database=env.str("DB_DATABASE")
)

Connected to: 8.1.0
You are connected to the database: ('mysql',)
-----------------------------------------------



### Create tables

In [5]:
migrator = Migrator(database)

In [6]:
migrator.migrate()

--------------------------------Starting migrate--------------------------------
Found migration files:  ['001_users.sql', '002_activities.sql', '003_trackpoints.sql']
Running migration:   001_users.sql       ✅
Running migration:   002_activities.sql  ✅
Running migration:   003_trackpoints.sql ✅
-------------------------Finished migrate in 11.855 ms--------------------------



### (DANGER) Wipe the DB
🚨 THIS WILL WIPE ALL DATA IN THE TABLES 🚨

In [7]:
# migrator.wipe()

### Seed Database from Data Set

In [8]:
migrator.seed_users()

------------------------------Starting seed_users-------------------------------
Seeded [(182,)] Users
------------------------Finished seed_users in 7.969 ms-------------------------



In [9]:
migrator.seed_activities()


----------------------------Starting seed_activities----------------------------
Generating seed data for user: 135	✅
Generating seed data for user: 132	✅
Generating seed data for user: 104	✅
Generating seed data for user: 103	✅
Generating seed data for user: 168	✅
Generating seed data for user: 157	✅
Generating seed data for user: 150	✅
Generating seed data for user: 159	✅
Generating seed data for user: 166	✅
Generating seed data for user: 161	✅
Generating seed data for user: 102	✅
Generating seed data for user: 105	✅
Generating seed data for user: 133	✅
Generating seed data for user: 134	✅
Generating seed data for user: 160	✅
Generating seed data for user: 158	✅
Generating seed data for user: 167	✅
Generating seed data for user: 151	✅
Generating seed data for user: 169	✅
Generating seed data for user: 156	✅
Generating seed data for user: 024	✅
Generating seed data for user: 023	✅
Generating seed data for user: 015	✅
Generating seed data for user: 012	✅
Generating seed data for user: 

In [10]:
migrator.seed_track_points()

---------------------------Starting seed_track_points---------------------------
Generating seed data for user: 135	✅
Generating seed data for user: 132	✅
Generating seed data for user: 104	✅
Generating seed data for user: 103	✅
Generating seed data for user: 168	✅
Generating seed data for user: 157	✅
Generating seed data for user: 150	✅
Generating seed data for user: 159	✅
Generating seed data for user: 166	✅
Generating seed data for user: 161	✅
Generating seed data for user: 102	✅
Generating seed data for user: 105	✅
Generating seed data for user: 133	✅
Generating seed data for user: 134	✅
Generating seed data for user: 160	✅
Generating seed data for user: 158	✅
Generating seed data for user: 167	✅
Generating seed data for user: 151	✅
Generating seed data for user: 169	✅
Generating seed data for user: 156	✅
Generating seed data for user: 024	✅
Generating seed data for user: 023	✅
Generating seed data for user: 015	✅
Generating seed data for user: 012	✅
Generating seed data for user: 

## Part 2

In [80]:
tasks = Task(database)

### Task 1

In [36]:
tasks.task1()

---------------------------------Starting task1---------------------------------
-------------------------Finished task1 in 2296.411 ms--------------------------



,# Users,# Activities,# TrackPoints
0,182,16048,29045268


### Task 2

In [37]:
tasks.task2()

---------------------------------Starting task2---------------------------------
-------------------------Finished task2 in 12485.962 ms-------------------------



,Avg,Max,Min
0,167892,3030975,51


### Task 3

In [38]:
tasks.task3()

---------------------------------Starting task3---------------------------------
---------------------------Finished task3 in 6.707 ms---------------------------



,UserId,ActivityCount
0,128,2102
1,153,1793
2,025,715
3,163,704
4,062,691
5,144,563
6,041,399
7,085,364
8,004,346
9,140,345


### Task 4

In [47]:
tasks.task4()

---------------------------------Starting task4---------------------------------
--------------------------Finished task4 in 12.148 ms---------------------------



,UserId
0,010
1,020
2,052
3,062
4,068
5,073
6,080
7,081
8,084
9,085


### Task 5

In [85]:
tasks.task5()

---------------------------------Starting task5---------------------------------
Running statement:
 
            SELECT DISTINCT u.id AS UserID, Count(DISTINCT a.transportation_mode) as '# Transportation Modes', GROUP_CONCAT(DISTINCT a.transportation_mode SEPARATOR ', ') AS 'Transportation Modes'
            FROM Activities AS a
            LEFT JOIN Users AS u
                ON a.user_id = u.id
            WHERE a.transportation_mode != ""
            GROUP BY u.id
            ORDER BY Count(DISTINCT a.transportation_mode) DESC
            LIMIT 10;
        
---------------------------Finished task5 in 4.808 ms---------------------------



[autoreload of tasks failed: Traceback (most recent call last):
  File "/Users/larswaage/Code/large-distributed-data/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/larswaage/Code/large-distributed-data/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/larswaage/Code/large-distributed-data/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/larswaage/Code/large-distributed-data/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/larswaage/Code/large-distributed-data/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b

,UserID,# Transportation Modes,Transportation Modes
0,128,9,"airplane, bike, boat, bus, car, subway, taxi, ..."
1,062,9,"bike, boat, bus, car, run, subway, taxi, train..."
2,085,4,"bus, subway, taxi, walk"
3,065,4,"bike, subway, taxi, walk"
4,112,3,"bike, bus, walk"
5,078,3,"subway, taxi, walk"
6,080,3,"bike, bus, taxi"
7,081,3,"bike, bus, walk"
8,111,3,"bike, car, taxi"
9,010,3,"bus, taxi, train"
